In [1]:
import os
# import imagesize

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm
from dataset_utilits import load_imagesPath_ages_sex

In [2]:
train_truth = pd.read_csv('./data.nosync/train/train.csv')
validation_truth = pd.read_csv('./data.nosync/validation/validation.csv')
test_truth = pd.read_excel('./data.nosync/test/test.xlsx')

In [3]:
path_data_dir = 'data.nosync'
full_path = os.path.join(os.getcwd(), path_data_dir)
X_train, age_train, sex_train, X_validation, age_validation, sex_validation, X_test, age_test, sex_test = load_imagesPath_ages_sex(
    full_path, train_truth, validation_truth, test_truth, None)

age_train = [x / 200. for x in age_train]
age_validation = [x / 200. for x in age_validation]
age_test = [x / 200. for x in age_test]

age_train_df = pd.DataFrame(age_train, columns=['age'])
age_validation_df = pd.DataFrame(age_validation, columns=['age'])
age_test_df = pd.DataFrame(age_test, columns=['age'])

age_train_df = age_train_df.astype('float32')
age_validation_df = age_validation_df.astype('float32')
age_test_df = age_test_df.astype('float32')

prendo le image_names da  /Users/pietroferrazzi/Desktop/uni/human_data/project/HDA-bone-age-prediction/data.nosync/train/cut
prendo le image_names da  /Users/pietroferrazzi/Desktop/uni/human_data/project/HDA-bone-age-prediction/data.nosync/validation/cut
prendo le image_names da  /Users/pietroferrazzi/Desktop/uni/human_data/project/HDA-bone-age-prediction/data.nosync/test/cut


In [9]:
def process_image_2(path, label):
    # Desired size
    # num_row = IMG_SHAPE[1]
    # num_col = IMG_SHAPE[0]
    size = 299

    # Get the image
    img = tf.io.read_file(path)
    # Decode the PNG
    img = tf.image.decode_png(img)
    # Resize image
    img = tf.image.resize(img, (size, size))
    # Reshape image (this is not necessary but I do it so that I don't need to be modifying the shape in the input layer)
    #img = tf.reshape(img, [size, size, 3])
    img = tf.concat((img,img,img), axis=2)
    # Cast image to float32
    img = tf.cast(img, tf.float32)
    # Normalize image
    img = img / 255.0

    return img, label

def create_dataset_images(img_paths, ages, batch_size, shuffle, cache_file=None, repeat=True):
    # Create a Dataset object
    dataset = tf.data.Dataset.from_tensor_slices((img_paths, ages)).map(process_image_2)

    # Cache dataset
    if cache_file:
        dataset = dataset.cache(cache_file)

    # Shuffle
    if shuffle:
        dataset = dataset.shuffle(len(img_paths))

    # Repeat the dataset indefinitely
    if repeat:
        dataset = dataset.repeat()

    # Batch
    dataset = dataset.batch(batch_size=batch_size)

    # Prefetch
    dataset = dataset.prefetch(buffer_size=1)

    return dataset


batch_size = 8
train_dataset = create_dataset_images(img_paths=X_train,  # Only gender
                                      # second_branch_df=second_branch_df_train,   # Full features
                                      ages=age_train_df,
                                      batch_size=batch_size,
                                      shuffle=False)

validation_dataset = create_dataset_images(img_paths=X_validation, # Only gender
                                           # second_branch_df=second_branch_df_val,  # Full features
                                           ages=age_validation_df,
                                           batch_size=batch_size,
                                           shuffle=False)

train_steps = int(np.ceil(len(X_train) / batch_size))
validation_steps = int(np.ceil(len(X_validation) / batch_size))

In [16]:
resnet = tf.keras.applications.resnet.ResNet152(
    include_top=False,
    weights = None,
)

for layer in resnet.layers[150:]:
  layer.trainable = True

resnet.compile(optimizer='adam', loss='mean_absolute_error')

resnet.fit(train_dataset,
                         steps_per_epoch = train_steps,
                         epochs = 30,
                         validation_data = validation_dataset,
                         validation_steps = validation_steps)
  

Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/mean_absolute_error/sub/BroadcastGradientArgs' defined at (most recent call last):
    File "/opt/homebrew/anaconda3/envs/hda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/homebrew/anaconda3/envs/hda/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/homebrew/anaconda3/envs/hda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/homebrew/anaconda3/envs/hda/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/opt/homebrew/anaconda3/envs/hda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/homebrew/anaconda3/envs/hda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
 

In [6]:
def extract_features(path):
    image = cv2.imread(path, 0)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = cv2.resize(image, (224, 224)).reshape(-1, 224, 224, 3)
    IMG_SHAPE = image.shape()

    image = tf.keras.applications.resnet.preprocess_input(image)

    features = resnet.predict(image)[0]
    # maxs = np.max(features, axis=(0, 1))
    # return  features.reshape(100352, order='F'), maxs
    return  features

In [7]:
folders = [
    ('./data.nosync/test/images', './data.nosync/test/features.csv', './data.nosync/test/maxes.csv'),
    ('./data.nosync/validation/images', './data.nosync/validation/features.csv', './data.nosync/validation/maxes.csv'),
    ('./data.nosync/train/images', './data.nosync/train/features.csv', './data.nosync/train/maxes.csv'),
]

# features = []
maxes = []

for folder, feat_path, max_path in folders:
    images = os.listdir(folder)
    for image in tqdm(images):
        path = os.path.join(folder, image)
        image_maxes = extract_features(path)
        # features.append(image_features)
        maxes.append(image_maxes)
    # i_df = pd.DataFrame(features)
    # i_df.insert(0, 'image', images)
    # i_df.to_csv(feat_path, index=False)

    m_df = pd.DataFrame(maxes)
    m_df.insert(0, 'image', images)
    m_df.to_csv(max_path, index=False)

    maxes = []
    # features = []

  0%|          | 0/200 [00:00<?, ?it/s]

1/1 [==============================] - 0s 151ms/step


KeyboardInterrupt: 